In [ ]:
#df.to_excel('Lu_177.xls')
#df = pd.read_excel('Lu_177.xls', index_col = 'WOS_ID')

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_excel('Lu177-1000.xls')
df2 = pd.read_excel('Lu177-1999.xls')
df3 = pd.read_excel('Lu177-2999.xls')
df4 = pd.read_excel('Lu177-3999.xls')
df5 = pd.read_excel('Lu177-4999.xls')
df6 = pd.read_excel('Lu177-5432.xls')
df = pd.concat([df1,df2,df3,df4,df5,df6], axis = 0)

#### Remove fields that have virtualy no data or are irrelvant

In [3]:
void_fields = ['Unnamed: 69','Highly Cited Status','Date of Export', 'Hot Paper Status',
              '180 Day Usage Count','Publication Type','eISSN', 'ISBN', 'Book Authors', 'Book Author Full Names',
               'Book Series Subtitle', 'Language', 'Conference Location', 'Conference Sponsor','Publisher City',
               'Publisher','Since 2013 Usage Count','Cited Reference Count', 'Conference Host', 'Affiliations',
               'Author Keywords', 'Keywords Plus','Cited References','Open Access Designations','IDS Number','Web of Science Index',
               'Early Access Date', 'Number of Pages', 'Publisher Address','ISSN','Journal ISO Abbreviation', 'Book DOI',
              'Times Cited: All Databases']
for col in df.columns:
    if col in void_fields:
        df.drop(col, axis = 1, inplace=True)

#### Change Index to unique index provided by WoS

In [4]:
df.drop_duplicates('UT (Unique WOS ID)', inplace = True)
df['WOS_ID'] = df['UT (Unique WOS ID)'].str.slice(4)
df.set_index('WOS_ID', inplace = True)
df.drop('UT (Unique WOS ID)', axis = 1, inplace = True)

#### Get rid of spaces in the column names to use them as parameters

In [5]:
col_new = []
for col in df.columns:
    col_n = col.replace(' ','_')
    col_n = col_n.replace(',','')
    col_new.append(col_n)
df.columns = col_new

#### Drop Astronomy, Geo, Agriculture

In [7]:
dict_stop = ['plant', 'astro', 'geo', 'food', 'soil','agriculture', 'horticulture', 'environmental', 'communication', 'mathematics', 'optics' ]
for cat in dict_stop:
    df.drop(df[df.WoS_Categories.str.contains(cat)].index, inplace = True)
    df.drop(df[df.WoS_Categories.str.contains(cat.title())].index, inplace = True)
    df.drop(df[df.WoS_Categories.str.contains(cat.upper())].index, inplace = True)
    df.drop(df[df.Research_Areas.str.contains(cat)].index, inplace = True)
    df.drop(df[df.Research_Areas.str.contains(cat.title())].index, inplace = True)
    df.drop(df[df.Research_Areas.str.contains(cat.upper())].index, inplace = True)

    
# Drop all papers related to Hf and its isotopes. Sems to be exclusievely inorganic chem or soil science
Hf_lst = ['HF-177', 'Hf-177', 'Lu-Hf', 'afnium', 'zircon','photoionisation' ]
abs_hf = False
ttl_hf = False
for word in Hf_lst:
    abs_hf = abs_hf | df.Abstract.str.contains(word)
    ttl_hf = ttl_hf | df.Article_Title.str.contains(word)
ttl_notempty  = df.Article_Title.notnull() 
abs_notempty  = df.Abstract.notnull() 
df.drop(df[ttl_notempty & ttl_hf].index, inplace = True)
df.drop(df[abs_notempty & abs_hf].index, inplace = True)

#Also drop tose with no publication year
df.drop(df[df.Publication_Year.isnull()].index, inplace = True)

/tmp/ipykernel_9465/810638588.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.drop(df[abs_notempty & abs_hf].index, inplace = True)


In [9]:
df.to_excel('Lu_177.xls')

/tmp/ipykernel_9465/78824525.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel('Lu_177.xls')
